In [1]:
import os
import sys
import itertools #import combinations ### added
from collections import namedtuple
Item = namedtuple("Item", ['index', 'value', 'weight'])

In [2]:
data_folder = 'C:/D/coursera/discrete_opt/knapsack/data'
data_files = None
for dirpath, dirnames, filenames in os.walk(data_folder):
    data_files = [os.path.join(dirpath, fname) for fname in filenames]
    
filenames = list(reversed(data_files))
filenames[:3]

['C:/D/coursera/discrete_opt/knapsack/data\\ks_lecture_dp_3.txt',
 'C:/D/coursera/discrete_opt/knapsack/data\\ks_lecture_dp_2',
 'C:/D/coursera/discrete_opt/knapsack/data\\ks_lecture_dp_1']

In [3]:
class Node:
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to its children. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""    
    
    def __init__(self, parent = None, x= None, index= None, value= None, estimate= None, room= None):
        """
            action: a list for valid children
        """ 
        self.x = x # decision variable
        self.index = index # item index
        self.value = value
        self.estimate = estimate
        self.room = room 

        self.state =''.join([str(self.index), str(self.x)])
        self.parent = parent
        self.depth = 0
        if parent:
            self.depth = parent.depth+1    
            self.state = parent.state + self.state
            
    def __repr__(self):
        """Use by calling repr(node)"""
        return "Node(" + repr(self.index) + "," + repr(self.x) + "," +repr(self.value) + ","\
    + repr(self.estimate) + "," + repr(self.room) + ")" 
            
    def __lt__(self, node):
        return self.estimate < node.value
    
    def expand(self, items, fractions):
        """
            fractions is 1 by len(items) containing fractions of item.values, e.g.,
            1 for select all item and 0.2 for select 0.2 of the items weight and value
        """
        "List the nodes reachable in one step from this node."
        if self.index >= len(items)-1:
            return None
        
        children = []
        for x in range(2):
            if x == 0: # not select this child
                # assuming item[0].index 0 corresponds to fractions[0]
                estimate = self.estimate - fractions[self.index+1]*items[self.index+1].value
                child = self.child_node(x, self.index+1, self.value, estimate, self.room) 
                children.append(child)
            elif self.room-items[self.index+1].weight >=0: # select this child if it can fit
                child = self.child_node(x, self.index+1, 
                                self.value+items[self.index+1].value,
                                self.estimate, self.room-items[self.index+1].weight)
                children.append(child)
                
        return children
    
    def child_node(self,  x, index, value, estimate, room):
        return Node(self, x, index, value, estimate, room)
    
    def path(self):
        "Return a loist of nodes forming the path from the root to this node"
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def solution(self):
        "Return items selected along the path from root to this node"
        return self.value, [node.x for node in self.path()[1:]]
    def __eq__(self, other):
        return isinstance(other, Node) and self.x == other.x and self.index == other.index
    def __has__(self):
        return hash((self.x, self.index))

In [4]:
def depth_first_search(fractions, items, capacity, estimate):
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    If two paths reach a state, only use the first one. [Figure 3.7]"""
    
    root = Node(parent = None, x= None, index= -1, value= 0, estimate= estimate, room= capacity)
    frontier = [root]
    explored = set()
    best_node = None
    while frontier:
        node = frontier.pop()
        #print('\n', repr(node))
        
        if best_node and float(best_node.value) >= float(node.estimate):
            continue 
        if not best_node or node.value > best_node.value:
            best_node = node
        explored.add(node.state)
        #best_value = node.value 
        children = node.expand(items, fractions)
        if children:
            frontier.extend(child for child in children
                            if child.state not in explored and
                            child not in frontier)
    return best_node

In [5]:
def solve_bb_dfs(input_data):
    # Modify this code to run your optimization algorithm
    
    def compute_fractions(value_densities, capacity, items):
        
        sorted_indices = sorted(range(len(value_densities)), 
                                key=lambda k: value_densities[k], reverse=True)
        fractions, k = [0]*len(items), capacity
        estimate = 0
        for idx in sorted_indices:
            if k - items[idx].weight >= 0:
                fractions[idx] = 1
                estimate = estimate + items[idx].value
            elif k > 0:
                fraction = float(k)/float(items[idx].weight) 
                fractions[idx] = fraction
                estimate = estimate + fraction*items[idx].value
            k = k-items[idx].weight
            
        return estimate, fractions
    
    # parse the input
    lines = input_data.split('\n')
    firstLine = lines[0].split()
    item_count = int(firstLine[0])
    capacity = int(firstLine[1])

    items = []
    value_densities = []
    for i in range(1, item_count+1):
        line = lines[i]
        parts = line.split()
        items.append(Item(i-1, int(parts[0]), int(parts[1])))
        value_densities.append(float(parts[0])/float(parts[1]))
    
    estimate, fractions = compute_fractions(value_densities, capacity, items)
    
    best_node = depth_first_search(fractions, items, capacity, estimate)
    best_value = best_node.solution()[0]
    best_x_path = best_node.solution()[1]+[0]*(len(items)-len(best_node.solution()[1]))

    # prepare the solution in the specified output format
    output_data = str(best_value) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, best_x_path))
    return output_data

In [ ]:
file_location = 'C:/D/coursera/discrete_opt/knapsack/data\\ks_4_0'
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
solve_bb_dfs(input_data)

'18 0\n1 1 0 0'

In [ ]:
file_location = 'C:/D/coursera/discrete_opt/knapsack/data\\ks_45_0'
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
solve_bb_dfs(input_data)

In [ ]:
file_location = 'C:/D/coursera/discrete_opt/knapsack/data\\ks_10000_0'
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
solve_bb_dfs(input_data)

In [ ]:
def solve_greedy_it(input_data):
    # Modify this code to run your optimization algorithm

    # parse the input
    lines = input_data.split('\n')
    print('Line: {}\n'.format(lines))
    firstLine = lines[0].split()
    item_count = int(firstLine[0])
    capacity = int(firstLine[1])

    items = []

    for i in range(1, item_count+1):
        line = lines[i]
        parts = line.split()
        items.append(Item(i-1, int(parts[0]), int(parts[1])))
    print('Items \n', items)

    # a trivial greedy algorithm for filling the knapsack
    # it takes items in-order until the knapsack is full
    value = 0
    weight = 0
    taken = [0]*len(items)

    for item in items:
        if weight + item.weight <= capacity:
            taken[item.index] = 1
            value += item.value
            weight += item.weight
    
    # prepare the solution in the specified output format
    output_data = str(value) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, taken))
    return output_data

In [ ]:
file_location = 'C:/D/coursera/discrete_opt/knapsack/data\\ks_lecture_dp_3.txt'
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
solve_greedy_it(input_data)